![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Hybrid queries with Redis

This notebook will cover implementing 3 different hybrid search techniques with Redis:

- Linear Combination with the aggregation API
- Client-Side Reciprocal Rank Fusion
- Client-Side Reranking with a cross encoder model

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/03_hybrid_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install Packages

In [ ]:
# NBVAL_SKIP
%pip install -q redis redisvl numpy sentence-transformers

### Data/Index Preparation

In this examples we will load a list of movie objects with the following attributes: `title`, `rating`, `description`, and `genre`. 

For the vector part of our vector search we will embed the description so that user's can search for movies that best match what they're looking for.

We will then populate a search index with the records and begin querying.

**If you are running this notebook locally**, FYI you may not need to perform this step at all.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client, load data, generate embeddings

In [2]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [3]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer

# load model for embedding our movie descriptions
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_text(model, text):
    return np.array(model.encode(text)).astype(np.float32).tobytes()

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Note: convert embedding array to bytes for storage in Redis Hash data type
movie_data = [
    {
        **movie,
        "vector": embed_text(model, movie["description"])
    } for movie in movies
]

In [6]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'vector': b'\x9bf|=\x0e`\n;"\x92\xb7;<\xcb~\xbd\xfad\xce\xbb\xc3\x16J=V\xa7?=\xedv\x95<d\xfa\x06\xbe\x14Y\xcf=(\x07p=?\xdb\r\xbd\x95\xf2H\xbdje\xc6<E\xdfa=z8\x16\xbc\x00\xd4\x13<>\xaa\x1c=\xfd\xee\x89<\xbd\xb0-<\x82\xb2\x9f\xbc[\x0b\xc3\xbd\x98NR=xl\xf7\xbcN>\x17\xbe#\x12\x05\xb99u\xbf<\xb0\xe0b\xba\xd3\xa6\xa8\xbdx\xdc\xec\xbcRc%=\xe4\xe7r\xbb\x1eOG=?(\x85=o@\xa2\xbc2Z\xd0\xbdC%K\xbd\xb9\xed\x94\xbcR\xddH=\x92&F<\xc6*\xec<\x90\xd8\x8d\xbd\xcbZ\x98<\t\xa3\xa3=>g3\xbd&\xcd\xbd\xbd\x95$\xf7;\xfd\xf4z=\xfc\xb4\x8c=\x85\x0e\xc6\xbdnI\x90\xbdJ\x16\xbd;s\xe7\x0c\xbd 3\xc9\xbc\x85\xf8\xbb\xbc\xbf&u\xbb5\x8f\xca<\x05\x80J=\x0f\xaf*=\x8bOU\xbd\xc8\xf0\x95\xbc\x1d\x02\x19=)\xf4K<\xcb\xc2\t=F\x83\xac=\x9f\xd7\xb8\xbd\xf2\xb5\x9c\xbdB\x85\x18=\x96d&=-3\xf8<\xfa\xf7\x88<\x16v\xf2\xbb-=[\xbd\xf7\xac\xee\xbb5:A\xbd\xd9d\x

## Define Redis index schema

In [7]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
  },
  "fields": [
    {
        "name": "title",
        "type": "text",
    },
    {
        "name": "description",
        "type": "text",
    },
    {
        "name": "genre",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "rating",
        "type": "numeric",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

11:43:01 redisvl.index.index INFO   Index already exists, overwriting.


In [8]:
index.info()

{'index_name': 'movies',
 'index_options': [],
 'index_definition': ['key_type',
  'HASH',
  'prefixes',
  ['rvl'],
  'default_score',
  '1'],
 'attributes': [['identifier',
   'title',
   'attribute',
   'title',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'description',
   'attribute',
   'description',
   'type',
   'TEXT',
   'WEIGHT',
   '1'],
  ['identifier',
   'genre',
   'attribute',
   'genre',
   'type',
   'TAG',
   'SEPARATOR',
   ',',
   'SORTABLE'],
  ['identifier',
   'rating',
   'attribute',
   'rating',
   'type',
   'NUMERIC',
   'SORTABLE',
   'UNF'],
  ['identifier',
   'vector',
   'attribute',
   'vector',
   'type',
   'VECTOR',
   'algorithm',
   'HNSW',
   'data_type',
   'FLOAT32',
   'dim',
   384,
   'distance_metric',
   'COSINE',
   'M',
   16,
   'ef_construction',
   200]],
 'num_docs': 0,
 'max_doc_id': 0,
 'num_terms': 0,
 'num_records': 0,
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.02034759521484375',
 'total_inverted_inde

### Populate index

In [9]:
index.load(movie_data)

['rvl:d0df27f540dd4052a983af63bdd8f9c2',
 'rvl:48d8fdf8a31a4b3ca5042a489d48caee',
 'rvl:ea71410253bc4db3af886d68e391cdef',
 'rvl:56e4fb3792a24972af7940fedd2f439b',
 'rvl:fdd3f2e4b9ac40169f13e0b66a8f95a7',
 'rvl:dc6e9b64a1fb4b709689832db38b1f4f',
 'rvl:03daf42e237e43c8b15e97178a9d77b2',
 'rvl:a98734e04762440f90023839610a8896',
 'rvl:2c22a85656af4f73ba8926f4da180b75',
 'rvl:4b6a40f1259c434582e839c6c4952215',
 'rvl:15b3d54fe2db4f94bf9d8cd149a09c12',
 'rvl:b6f0a94095304c4b93ad1625cdfb1316',
 'rvl:b87f1df18d8344b495d92e24dc3074e2',
 'rvl:02ebb1012a87433a9d5b321e052ca347',
 'rvl:0d587457f1724916ae1a1b59c5dabb1a',
 'rvl:b1d1f92e4f1b4198860d42281f221ffd',
 'rvl:57c037a836d449cf889fa34808731c71',
 'rvl:6895eadb2f65447f872bdc4b00b63acc',
 'rvl:219c4aef504e462494b0254e5983e0ee',
 'rvl:0a5a5ac12186464b9fd15ef4c958fc04']

# Linear Combination

Now that our index is populated and ready, let's execute our first hybrid search technique. This query will calculate a linear combination of the BM25 score for our provided text search and the cosine distance between vectors calculated via a KNN vector query.

This style of query is useful when you want to combine the power of key words (BM25) with the flexibility of vector search.

In [14]:
# import aggregations API
import redis.commands.search.aggregation as aggregations

# user vector query
user_query = "a movie with animals"
embedded_user_query = embed_text(model, user_query)

# @description will apply BM25 for the phrase "hero" according to the search
# Vector distance will be calculated between the index @vector and the input user_query
query_string = "(@description:family)=>[KNN 3 @vector $vec_param AS dist]"

# .scorer: specifies the scoring function to use either BM25 or TFIDF
# .add_scores: adds the scores to the result
# .apply: algebraic operation that makes sense for your use case
# .load: specifies fields to return - all in this case.
# .dialect: specifies the query dialect to use.

# TODO: scorer not in redis-py yet

req = aggregations\
        .AggregateRequest(query_string)\
        .add_scores()\
        .apply(hybrid_score="@__score + 2*@dist")\
        .load("title", "dist", "__score", "hybrid_score")\
        .dialect(4)

res = client.ft(index_name).aggregate(req, query_params={'vec_param': embedded_user_query})
res.rows

[[b'dist',
  b'0.842549920082',
  b'__score',
  b'1.5',
  b'title',
  b'The Incredibles',
  b'hybrid_score',
  b'3.18509984016'],
 [b'dist',
  b'0.91923135519',
  b'__score',
  b'1.5',
  b'title',
  b'Black Widow',
  b'hybrid_score',
  b'3.33846271038'],
 [b'dist',
  b'1.0012062788',
  b'__score',
  b'1.5',
  b'title',
  b'Gladiator',
  b'hybrid_score',
  b'3.5024125576']]